In [81]:
from sklearn.datasets import load_diabetes
import numpy as np
from  sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [82]:
x,y = load_diabetes(return_X_y=True)

In [83]:
y.shape

(442,)

In [84]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [85]:
lr= LinearRegression()
lr.fit(x_train,y_train)
lr.predict(x_test)
r2_score(y_test,lr.predict(x_test))

0.4526027629719196

In [86]:
print(lr.coef_)
print(lr.intercept_)

[  37.90402135 -241.96436231  542.42875852  347.70384391 -931.48884588
  518.06227698  163.41998299  275.31790158  736.1988589    48.67065743]
151.34560453985995


In [87]:
class GDregressor():
    def __init__(self,lr=0.01,epochs=100):
        self.coef_ = None
        self.intercept_ = None
        self.lr = lr
        self.epochs = epochs
    
    def fit(self,x_train,y_train):
        self.coef_ = np.ones(x.shape[1])
        self.intercept_=0
        for i in range(self.epochs):
            y_pre = np.dot(x_train,self.coef_) + self.intercept_
            intercept_der= -2*np.mean(y_train-y_pre)
            self.intercept_= self.intercept_ - (self.lr * intercept_der)
            coef_der = -2 * np.dot((y_train - y_pre),x_train)/x_train.shape[0]
            self.coef_ = self.coef_ - (self.lr * coef_der)
        print(self.coef_)
        print(self.intercept_)
    def predict(self,x):
        return np.dot(x,self.coef_) + self.intercept_


In [88]:
gd= GDregressor(lr=0.1,epochs=1000)
gd.fit(x_train,y_train)
pre= gd.predict(x_test)
r2_score(y_test,pre)

[  57.36300862  -43.56362668  270.30125536  192.94011506   27.79121735
    2.40698174 -146.62310137  136.73992976  224.37340174  135.00101101]
152.26172887146495


0.41666791172170736

In [89]:
gd.intercept_

152.26172887146495

## Stochastic gradient

In [90]:
class SGDregressor():
    def __init__(self,lr=0.01,epochs=100):
        self.coef_ = None
        self.intercept_ = None
        self.lr = lr
        self.epochs = epochs
    
    def fit(self,x_train,y_train):
        self.coef_ = np.ones(x.shape[1])
        self.intercept_=0
        for i in range(self.epochs):
            for j in range(x_train.shape[0]):
                idx= np.random.randint(x_train.shape[0])
                y_pre= np.dot(x_train[idx],self.coef_) + self.intercept_
                intercept_der= -2*(y_train[idx]-y_pre)
                self.intercept_= self.intercept_ - (self.lr * intercept_der)

                coef_der = -2 * np.dot((y_train[idx] - y_pre),x_train[idx])
                self.coef_ = self.coef_ - (self.lr * coef_der)
        print(self.coef_)
        print(self.intercept_)
    def predict(self,x):
        return np.dot(x,self.coef_) + self.intercept_

In [91]:
sgd= SGDregressor(lr=0.22,epochs=50)
sgd.fit(x_train,y_train)
pre= sgd.predict(x_test)
r2_score(y_test,pre)

[  33.79501842 -196.26351342  515.58489779  368.56014909 -223.76905217
  -31.07656984 -165.58170142  188.60922278  466.03390178   65.99560073]
125.16384428580216


0.28343804275897266

In [92]:
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
reg = make_pipeline(StandardScaler(),SGDRegressor())
reg.fit(x_train,y_train)
pre= reg.predict(x_test)
r2_score(y_test,pre)

0.455208970142971

## Mini-batch gradient descent

In [93]:
import random
class MBGDregressor():
    def __init__(self,batch_size,lr=0.01,epochs=100):
        self.coef_ = None
        self.intercept_ = None
        self.lr = lr
        self.batch_size = batch_size
        self.epochs = epochs
    
    def fit(self,x_train,y_train):
        self.coef_ = np.ones(x_train.shape[1])
        self.intercept_=0
        for i in range(self.epochs):
            for j in range(int(x_train.shape[0]/self.batch_size)):
                idx = random.sample(range(x_train.shape[0]),self.batch_size)
                y_pre= np.dot(x_train[idx],self.coef_) + self.intercept_
                intercept_der= -2*np.mean(y_train[idx]-y_pre)
                self.intercept_= self.intercept_ - (self.lr * intercept_der)

                coef_der = -2 * np.dot((y_train[idx] - y_pre),x_train[idx])
                self.coef_ = self.coef_ - (self.lr * coef_der)
        print(self.coef_)
        print(self.intercept_)
    def predict(self,x):
        return np.dot(x,self.coef_) + self.intercept_

In [94]:
mbgr = MBGDregressor(batch_size=int(x_train.shape[0]/50),lr=0.01,epochs=100)
mbgr.fit(x_train,y_train)

[  49.31231325 -166.4250745   464.07173651  292.47731916  -48.95181583
  -99.35720565 -212.85708295  151.3567671   355.81052305  136.58278089]
149.7653285694222


In [95]:
pre= mbgr.predict(x_test)
r2_score(y_test,pre)

0.4573420361975896

#### USING SKLEARN

In [98]:
sgd = SGDRegressor(learning_rate='constant',eta0=0.2)
batch_size = 35
for i in range(100):
    idx = random.sample(range(x_train.shape[0]),batch_size)
    sgd.partial_fit(x_train[idx],y_train[idx])

In [99]:
pre= sgd.predict(x_test)
r2_score(y_test,pre)

0.44142668648925243